In [1]:
%run classesDefinations.ipynb

In [2]:
import functools

import gymnasium
import numpy as np
from gymnasium.spaces import Discrete, Dict, Box
from gymnasium.wrappers import FlattenObservation #IMPORTANT USE IT TO FLATTEN OBS SPACE

from pettingzoo import AECEnv
from pettingzoo.utils import agent_selector, wrappers

import json
import random

In [3]:
NUMBEROFBUSES = 10
CAPACITY = 5
VEHILECORDSLAT = 35.8942679
VEHILECORDSLOG = 14.5086503 #CORDS OF THE 15 VALLETTA BUS BAY

dataFilePath = "./Data/BusStopsMalta/export.json"

In [4]:
from datetime import datetime, timedelta

def divide_time_interval(n):
    """
    Divides the time interval between 8:00 AM and Midnight into 'n' equal intervals.

    Parameters:
    n (int): Number of intervals to divide the time into.

    Returns:
    list: A list of time intervals.
    """
    
    # Define start and end times
    start_time = datetime.strptime("08:00 AM", "%I:%M %p")
    end_time = datetime.strptime("12:00 AM", "%I:%M %p")

    # Calculate total duration in seconds
    total_duration = (end_time - start_time).seconds

    # Calculate duration of each interval in seconds
    interval_duration = total_duration / n

    # Generate time intervals
    intervals = []
    for i in range(n):
        interval_start = start_time + timedelta(seconds=i * interval_duration)
        intervals.append(interval_start.time())

    return intervals

In [7]:
class BusRoutingHandler():
    def __init__(self, vehileAmount = 10, numberOfRequest = 50):
        self.vehiles = self.initVehiles(vehileAmount)
        self.requests = self.initRequests(numberOfRequest)
    
    def initVehiles(self, vehileAmount):
        vehileList = []
        for i in range(vehileAmount):
            vechCords = Cords(VEHILECORDSLAT, VEHILECORDSLOG)
            vehileList.append(Vehicle(CAPACITY, vechCords))
        return vehileList

    def initRequests(self, numberOfRequest):
        timeIntervals = divide_time_interval(numberOfRequest)
        f = open(dataFilePath, "r")
        data = json.load(f)
        elements = data["elements"]
        elementsLen = len(elements)
        request = []
        time = 0
        for i in range(numberOfRequest):
            #Get two random indexes from all the bus stops 
            r1 = random.randint(0, elementsLen-1)
            r2 = random.randint(0, elementsLen-1)
            while r1 == r2:
                r2 = random.randint(0, elementsLen-1)
            
            #Get the coordinates of the two bus stops
            lat1 = elements[r1]["lat"]
            lon1 = elements[r1]["lon"]
            lat2 = elements[r2]["lat"]
            lon2 = elements[r2]["lon"]

            #Create the request Cords
            reqPickup = RequestCords(lat1, lon1, True)
            reqDropoff = RequestCords(lat2, lon2, False)

            #Create the request
            time += 20
            request.append(Request(random.randint(1,3), reqPickup, reqDropoff, timeIntervals[i]))
        f.close()
        return request

    def getVehiles(self):
        pass

    def getRequests(self):
        pass

busRouting = BusRoutingHandler()
print(busRouting.vehiles[2])

Vehicle: id: 2125993963152, capacity: 5, currentPosition: lat: 35.8942679,long: 14.5086503, route: 0


In [6]:
def env(render_mode=None): #Petting Zoo Env
    """
    The env function often wraps the environment in wrappers by default.
    You can find full documentation for these methods
    elsewhere in the developer documentation.
    """

    internal_render_mode = render_mode if render_mode != "ansi" else "human"
    env = raw_env(render_mode=internal_render_mode)

    # This wrapper is only for environments which print results to the terminal
    if render_mode == "ansi":
        env = wrappers.CaptureStdoutWrapper(env)

    # this wrapper helps error handling for discrete action spaces
    env = wrappers.AssertOutOfBoundsWrapper(env)

    # Provides a wide vareity of helpful user errors
    # Strongly recommended
    env = wrappers.OrderEnforcingWrapper(env)

    return env

class raw_env(AECEnv):
    """
    The metadata holds environment constants. From gymnasium, we inherit the "render_modes",
    metadata which specifies which modes can be put into the render() method.
    At least human mode should be supported.
    The "name" metadata allows the environment to be pretty printed.
    """

    metadata = {"render_modes": ["human"], "name": "Flexible_Bus"}

    def __init__(self, render_mode=None):
        """
        The init method takes in environment arguments and
         should define the following attributes:
        - possible_agents
        - render_mode

        Note: as of v1.18.1, the action_spaces and observation_spaces attributes are deprecated.
        Spaces should be defined in the action_space() and observation_space() methods.
        If these methods are not overridden, spaces will be inferred from self.observation_spaces/action_spaces, raising a warning.

        These attributes should not be changed after initialization.
        """

        self.agents = ["bus_" + str(r) for r in range(NUMBEROFBUSES)]
        self.possible_agents = self.agents[:]

        #Set up action space and observation space
        self._action_spaces = {
            agent: Dict({
                        "accept_request": Discrete(2),
                        "insert_request": Discrete(CAPACITY), #Require Maskings
                         }) for agent in self.possible_agents
        }
        
        self._observation_spaces = {
            agent: Dict({
                "observation": Dict(
                    { #Simple Defination containing the bus position cordinates and the bus capacity
                    "bus_position": Box(low=0, high=90, shape=(2,), dtype=np.float32),
                    "bus_capacity": Discrete(CAPACITY),
                    "request_position":Box(low=0, high=90, shape=(2,), dtype=np.float32),
                    }),
                "action_mask": Dict(
                    { #Action mask needed to not allow the bus to exceed capacity
                    "accept_request": Box(low=0, high=1, shape=(2,), dtype=np.int8),
                    "insert_request": Box(low=0, high=1, shape=(2,), dtype=np.int8),
                    })
            }) for agent in self.possible_agents
        } 

        self.render_mode = render_mode

    # Observation space defination
    def observation_space(self, agent):
        return self.observation_spaces[agent]

    # Action space defination
    def action_space(self, agent):
        return self.action_spaces[agent]

    def render(self):
        """
        Renders the environment. In human mode, it can print to terminal, open
        up a graphical window, or open up some other display that a human can see and understand.
        """
        return None

    def observe(self, agent):
        """
        Observe should return the observation of the specified agent. This function
        should return a sane observation (though not necessarily the most up to date possible)
        at any time after reset() is called.
        """

        #Change CODE to reflect agents values
        observation = {
            "bus_position": np.zeros(2,dtype=np.float32), 
            "bus_capacity": 0,
            "request_position": np.zeros(2,dtype=np.float32),
        }

        #Change CODE to reflect agents values
        action_mask = {
            "accept_request": np.zeros(2,np.int8),
            "insert_request": np.zeros(5,np.int8)
        }

        return {"observation": observation, "action_mask": action_mask}

    def close(self):
        """
        Close should release any graphical displays, subprocesses, network connections
        or any other environment data which should not be kept around after the
        user is no longer using the environment.
        """
        pass

    def reset(self, seed=None, options=None):
        """
        Reset needs to initialize the following attributes
        - agents
        - rewards
        - _cumulative_rewards
        - terminations
        - truncations
        - infos
        - agent_selection
        And must set up the environment so that render(), step(), and observe()
        can be called without issues.
        Here it sets up the state dictionary which is used by step() and the observations dictionary which is used by step() and observe()
        """
        self.agents = self.possible_agents[:]
        self.rewards = {agent: 0 for agent in self.agents}
        self._cumulative_rewards = {agent: 0 for agent in self.agents}
        self.terminations = {agent: False for agent in self.agents}
        self.truncations = {agent: False for agent in self.agents}
        self.infos = {agent: {} for agent in self.agents}

        #Self.state missing?
        
        """
        Selects the first Bus
        """
        self._agent_selector = agent_selector(self.agents)
        self.agent_selection = self._agent_selector.next()

    #Requries Definations
    def step(self, action):
        """
        step(action) takes in an action for the current agent (specified by
        agent_selection) and needs to update
        - rewards
        - _cumulative_rewards (accumulating the rewards)
        - terminations
        - truncations
        - infos
        - agent_selection (to the next agent)
        And any internal state used by observe() or render()
        """
        if (
            self.terminations[self.agent_selection]
            or self.truncations[self.agent_selection]
        ):
            # handles stepping an agent which is already dead
            # accepts a None action for the one agent, and moves the agent_selection to
            # the next dead agent,  or if there are no more dead agents, to the next live agent
            self._was_dead_step(action)
            return

        agent = self.agent_selection

        # the agent which stepped last had its _cumulative_rewards accounted for
        # (because it was returned by last()), so the _cumulative_rewards for this
        # agent should start again at 0
        self._cumulative_rewards[agent] = 0

        # stores action of current agent
        self.state[self.agent_selection] = action

        # collect reward if it is the last agent to act
        if self._agent_selector.is_last():
            # rewards for all agents are placed in the .rewards dictionary
            self.rewards[self.agents[0]], self.rewards[self.agents[1]] = REWARD_MAP[
                (self.state[self.agents[0]], self.state[self.agents[1]])
            ]

            self.num_moves += 1
            # The truncations dictionary must be updated for all players.
            self.truncations = {
                agent: self.num_moves >= NUM_ITERS for agent in self.agents
            }

            # observe the current state
            for i in self.agents:
                self.observations[i] = self.state[
                    self.agents[1 - self.agent_name_mapping[i]]
                ]
        else:
            # necessary so that observe() returns a reasonable observation at all times.
            self.state[self.agents[1 - self.agent_name_mapping[agent]]] = NONE
            # no rewards are allocated until both players give an action
            self._clear_rewards()

        # selects the next agent.
        self.agent_selection = self._agent_selector.next()
        # Adds .rewards to ._cumulative_rewards
        self._accumulate_rewards()

        if self.render_mode == "human":
            self.render()